In [ ]:
!pip install -qU \
  langchain-core \
  langchain-community \
  langsmith \
  langchain_google_genai \
  langchain_community
!pip install -U langchain-google-genai langchain-core pydantic google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of goo

[Check tracing](https://smith.langchain.com/o/672a3466-58e2-4470-90c3-03df0e49da13/projects/p/dc91e544-64bf-4cb4-b6c6-0ac95c6df613?timeModel=%7B%22duration%22%3A%227d%22%7D)

In [ ]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-langsmith-openai"

Enter LangSmith API Key: ··········


In [ ]:
prompt = """
Answer the users query based on the context below.
If you cannot answer the question using the provided information just answer, i
dont know.

Context = {context}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",prompt),
        ("user","{query}")
    ]
)

In [ ]:
print(prompt_template.input_variables)

['context', 'query']


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(api_key=getpass("Enter Gemini API Key: "),temperature=0,model="gemini-2.0-flash")
llm.invoke("hello")

Enter Gemini API Key: ··········


AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7040d062-c59d-4d83-b67e-85c66f5ac309-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [ ]:
context = """
One Piece is a Japanese manga and anime series created by Eiichiro Oda. The story follows a young pirate named Monkey D. Luffy, who dreams of becoming the Pirate King by finding the legendary treasure known as One Piece, hidden somewhere in the vast sea called the Grand Line.
Luffy forms a crew called the Straw Hat Pirates, each member having a unique background and special abilities. Some key members include Roronoa Zoro, a swordsman; Nami, a navigator; Sanji, a cook; Usopp, a sniper; Tony Tony Chopper, a doctor; Nico Robin, an archaeologist; Franky, a shipwright; Brook, a musician; and Jinbe, a helmsman.
In the world of One Piece, there are mysterious fruits called Devil Fruits. Eating one grants a person special powers, such as Luffy's ability to stretch like rubber. However, it also takes away the ability to swim.
The world is divided among pirates, the World Government, and the Marines, who try to maintain control. Powerful pirates known as Yonkō and revolutionary forces also challenge the status quo.
One Piece is known for its epic adventures, deep world-building, strong themes of friendship and freedom, and complex characters. It combines action, comedy, and emotional storytelling over hundreds of episodes and chapters.
"""

query = "Difference between world gov and marines"

pipeline = (
            {
                "context":lambda x : x["context"],
                "query":lambda x : x["query"]
            }
           | prompt_template
           | llm
           | {"output":lambda x : x}
)

In [ ]:
pipeline.invoke({'context':context,"query":query})

{'output': AIMessage(content="The World Government and the Marines are distinct entities in the world of One Piece, but they are closely related. The World Government is the overarching global organization that governs most of the world and maintains order. The Marines are essentially the World Government's military force, tasked with enforcing the law, combating pirates, and maintaining peace (or what they perceive as peace) in the world.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--c94e86b5-831d-4d4d-8d7a-9df81cd5c4de-0', usage_metadata={'input_tokens': 309, 'output_tokens': 77, 'total_tokens': 386, 'input_token_details': {'cache_read': 0}})}